## Support mismatch

This notebook generates the data in the bar plots showing the support mismatch for the reconstructed coefficient matrices $C$ and the Kirchoff matrices $K$, using both the integral and differential forms. The least squares solver is STLS. 

The corresponding figures in the paper are 

* Figure 5 (M1 without noise): set `model = 'M1'`
* Figure 10 (M20 without noise): set `model = 'M20'`
* Figure 16 (Van de Vusse without noise): set `model = 'VdV'`

Each run of the notebook will save data in a dedicated subfolder. After this, the plots can be generated with `Support_mismatch_plotter.ipynb`.

# Data-driven discovery of chemical reaction networks: support mismatch tests

**TODO: remove comment on K support mismatch**

## Import modules and auxiliary functions

In [1]:
import os
from tqdm import tqdm
from multiprocessing import Pool
import time

import numpy as np

import scipy.integrate as integrate
from scipy.interpolate import make_interp_spline

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import mechanisms as mech
import graphsindy as gs

## Model parameters

In [2]:
# Define general parameters for each model:
# - min_k, max_k define the interval from which we uniformly pick the reaction rates
# - w is the number of experiments for each model for D, D_int to attain full-row rank
# - reduce_fn is the filtration scheme for the ODE system recovered by SINDy. 
#   reduce_model() removes all inactive columns from C; 
#   reduce_model2() removes all inactive columns from C starting from quadratic monomials
# - build is the constructor function of each model's class.

configs = {
    'M1': {
        'min_k': 5e-2, 'max_k': 1.0, 
        'w': 6,
        'reduce_fn': gs.reduce_model,
        'build': lambda rng, p, mech: mech.M1(
            rng.uniform(p['min_k'], p['max_k'], size=4)
        )
    },
    'M20': {
        'min_k': 5e-2, 'max_k': 1.0,
        'w': 8,
        'reduce_fn': gs.reduce_model2,
        'build': lambda rng, p, mech: mech.M20(
            rng.uniform(p['min_k'], p['max_k'], size=6)
        )
    },
    'VdV': {
        'min_k': 5e-2, 'max_k': 1.0, 
        'w': 4,
        'reduce_fn': gs.reduce_model2,
        'build': lambda rng, p, mech: mech.VdV(
            rng.uniform(p['min_k'], p['max_k'], size=3)
        )
    }
}

## Simulation parameters

In [3]:
num_trials = 1000                                      # number of comparison trials
main_rng = np.random.default_rng()                     # or any fixed seed for reproducibility (420, )
seeds = main_rng.integers(0, 1e6, size=num_trials)     # generate array of seeds to be used in the trials

# Choose model
model = 'M1'
#model = 'M20'
#model = 'VdV'
#model = 'Szederkenyi'

# Single-run parameters
tmax, npts = 20, 25                 
t_eval = np.linspace(0, tmax, npts)
rtol = 1e-10

# Interpolation parameters
spline_deg  = 3
spline_type = 'not-a-knot'

In [4]:
# Create/open directory for saving trials data
# Parent directory — shared by all runs of same model
parent_folder = f"{model}_supmis"
os.makedirs(parent_folder, exist_ok=True)

# Individual run directory
run_folder = f"npts-{npts}"
SAVE_DIR = os.path.join(parent_folder, run_folder)
#os.makedirs(SAVE_DIR, exist_ok=True)

In [5]:
# Storage for metrics
support_mismatches_Cint = []
support_mismatches_Cdif = []

support_mismatches_Kint = []
support_mismatches_Kdif = []

In [6]:
def run_trial(trial):
    """
    Run one trial and return all support mismatch metrics.
    """
    try:
        rng = np.random.default_rng(seed=seeds[trial])

        # Extract model parameters
        params = configs[model]
        w = params['w']
        Model = params['build'](rng, params, mech)

        M = Model.M
        N = Model.N
        constants = Model.Constants()
        C_ex = Model.ExactCoeffMatrix()
        K_ex = Model.ExactKirchhoffMatrix()

        # Initial conditions
        initial_values = rng.random((w, M))

        # Simulate ground truth
        X = np.zeros((M, 0))
        for i in range(w):
            sol = integrate.solve_ivp(fun=lambda t, U: gs.matrix_ODE(t, U, C_ex),
                                      t_span=t_eval[[0, -1]],
                                      y0=initial_values[i],
                                      t_eval=t_eval, rtol=rtol)
            X = np.hstack((X, sol.y))

        # SINDy
        K, L = gs.build_int_dif_matrix(t_eval, w, spline_deg, spline_type)
        D, D_labels = gs.build_dictionary(Model.labels, X)
        IVP = gs.build_ivp_matrix(npts, X)
        D_int = D @ K
        X_0 = X - IVP
        X_dif = X @ L

        threshold = np.min(constants) / 2
        C_int = gs.itls(D_int, X_0, threshold)
        C_dif = gs.itls(D, X_dif, threshold)

        # Prepare output container
        results = {"Cint": gs.support_mismatch(C_int, C_ex),
                   "Cdif": gs.support_mismatch(C_dif, C_ex),
                   "Kint": None,
                   "Kdif": None}

        reduce_fn = configs[model]['reduce_fn']

        def Kirchhoff_Analysis(C, M, N, D_labels, K_ex, threshold):
            C_red, Q, Q_labels = reduce_fn(C, M, N, D_labels)
            K = gs.solve_kirchhoff(Q, C_red)
            K = np.where(np.abs(K) >= threshold, K, 0)
            return gs.support_mismatch(K, K_ex)

        # Kirchhoff comparisons
        if gs.support_mismatch(C_int, C_ex) == 0:
            results["Kint"] = Kirchhoff_Analysis(C_int, M, N, D_labels, K_ex, threshold)
        if gs.support_mismatch(C_dif, C_ex) == 0:
            results["Kdif"] = Kirchhoff_Analysis(C_dif, M, N, D_labels, K_ex, threshold)

        return results

    except Exception as e:
        print(f"Trial {trial} failed: {e}")
        return None

In [7]:
if __name__ == "__main__":
    start = time.time()
    # Number of cores to use
    NUM_WORKERS = 30
    all_results = []

    with Pool(processes=NUM_WORKERS) as pool:
        for result in tqdm(pool.imap_unordered(run_trial, range(num_trials)), total=num_trials):
            if result is not None:
                all_results.append(result)

    print(f"All done in {time.time() - start:.2f} seconds.")

  0%|          | 1/1000 [00:00<01:54,  8.70it/s]

Trial 21 failed: operands could not be broadcast together with shapes (5,5) (3,3) Trial 24 failed: operands could not be broadcast together with shapes (5,5) (3,3) 

Trial 27 failed: operands could not be broadcast together with shapes (8,8) (3,3) 
Trial 3 failed: operands could not be broadcast together with shapes (5,5) (3,3) 


  1%|          | 9/1000 [00:00<00:23, 42.69it/s]

Trial 16 failed: operands could not be broadcast together with shapes (5,5) (3,3) Trial 19 failed: operands could not be broadcast together with shapes (5,5) (3,3) 

Trial 11 failed: operands could not be broadcast together with shapes (4,4) (3,3) 
Trial 25 failed: operands could not be broadcast together with shapes (7,7) (3,3) 
Trial 35 failed: operands could not be broadcast together with shapes (4,4) (3,3) 
Trial 22 failed: operands could not be broadcast together with shapes (5,5) (3,3) 
Trial 32 failed: operands could not be broadcast together with shapes (4,4) (3,3) 
Trial 6 failed: operands could not be broadcast together with shapes (8,8) (3,3) 


  2%|▎         | 25/1000 [00:00<00:10, 88.97it/s]

...

Trial 965 failed: operands could not be broadcast together with shapes (12,12) (3,3) 
Trial 962 failed: operands could not be broadcast together with shapes (12,12) (3,3) 
Trial 991 failed: operands could not be broadcast together with shapes (9,9) (3,3) 
Trial 979 failed: operands could not be broadcast together with shapes (7,7) (3,3) 
Trial 987 failed: operands could not be broadcast together with shapes (7,7) (3,3) 
Trial 984 failed: operands could not be broadcast together with shapes (6,6) (3,3) 
Trial 971 failed: operands could not be broadcast together with shapes (7,7) (3,3) 
Trial 969 failed: operands could not be broadcast together with shapes (11,11) (3,3) 
Trial 996 failed: operands could not be broadcast together with shapes (5,5) (3,3) 
Trial 995 failed: operands could not be broadcast together with shapes (8,8) (3,3) 
Trial 990 failed: operands could not be broadcast together with shapes (6,6) (3,3) 
Trial 994 failed: operands could not be broadcast together with shapes

100%|█████████▉| 996/1000 [00:08<00:00, 137.20it/s]

Trial 999 failed: operands could not be broadcast together with shapes (5,5) (3,3) 
Trial 998 failed: operands could not be broadcast together with shapes (12,12) (3,3) 
Trial 997 failed: operands could not be broadcast together with shapes (13,13) (3,3) 
Trial 989 failed: operands could not be broadcast together with shapes (14,14) (3,3) 


100%|██████████| 1000/1000 [00:08<00:00, 118.32it/s]


All done in 8.88 seconds.


In [8]:
# Initialize empty lists
support_mismatches = {"Cint": [], "Cdif": [], "Kint": [], "Kdif": []}

# Fill from all_results
for res in all_results:
    for key in support_mismatches:
        if res[key] is not None:
            support_mismatches[key].append(res[key])

# Convert to numpy arrays
for key in support_mismatches:
    support_mismatches[key] = np.asarray(support_mismatches[key])

max_interest = 10

def make_hist_f(arr, max_interest):
    if len(arr) == 0:
        hist = np.zeros(max_interest + 2, dtype=int)
    else:
        if np.max(arr) < max_interest + 2:
            hist, _ = np.histogram(arr, bins=np.arange(0, max_interest + 2))
        else:
            hist, _ = np.histogram(arr, bins=np.arange(0, np.max(arr) + 1))
        hist = np.hstack((hist[:max_interest + 1], np.sum(hist[max_interest + 1:])))
    return hist

results = {}
methods = [""]

for m in methods:
    suffix = "" if m == "" else m
    results[f"C{suffix}dif_hist_f"] = make_hist_f(support_mismatches[f"C{suffix}dif"], max_interest)
    results[f"C{suffix}int_hist_f"] = make_hist_f(support_mismatches[f"C{suffix}int"], max_interest)
    results[f"K{suffix}dif_hist_f"] = make_hist_f(support_mismatches[f"K{suffix}dif"], max_interest)
    results[f"K{suffix}int_hist_f"] = make_hist_f(support_mismatches[f"K{suffix}int"], max_interest)

In [9]:
if npts < 1000:
    savefile = f"npts-0{npts}.npz"
else:
    savefile = f"npts-{npts}.npz"

save_path = os.path.join(parent_folder, savefile)
np.savez(save_path, **results)